## Training a word2vec

In [67]:
from gensim.models import Word2Vec
from collections import defaultdict
import numpy as np
import pickle

In [2]:
train =np.load("../data/train.npy")

In [68]:
min_c=3
paragraph = set(train[:,2])

all_context = []
word_freq=defaultdict(int)
front = "1"
for para in paragraph:
    p=[]
    #add start and end
    #p.append("<start>")
    for letter in para:
        p.append(letter)
        word_freq[letter]+=1
    #p.append("<end>")
    all_context.append(p)
Nu = "@@"
wordoffew=[]
for i,row in enumerate(all_context):
    for j,col in enumerate(row):
        if word_freq[col]<min_c:
            wordoffew.append(col)
            all_context[i][j]=Nu



TypeError: write() argument must be str, not bytes

In [73]:
w2v_config = [word_freq,min_c,Nu]

with open("model_para/word_freq.pk","wb") as pk:
    pickle.dump(w2v_config,pk)
    pk.close()

In [4]:
len(set(wordoffew))

1209

In [45]:
min_c=3
QA_word2vec=Word2Vec(all_context,iter=50,min_count=min_c,size=256,window=20,workers=16)

vocab = dict([(k, v.index) for k, v in QA_word2vec.wv.vocab.items()])
weight_matrix = QA_word2vec.wv.syn0 #word_to_vec

QA_word2vec.save("model_para/QA_word2vec.bin")

In [53]:
QA_word2vec.most_similar(positive=["漂","亮","貝"],negative=[])

[('彗', 0.31442469358444214),
 ('歇', 0.29723507165908813),
 ('繆', 0.2952030599117279),
 ('托', 0.2632206678390503),
 ('吁', 0.26204097270965576),
 ('偶', 0.2530558407306671),
 ('泣', 0.25270524621009827),
 ('星', 0.24661768972873688),
 ('譜', 0.2448553442955017),
 ('韋', 0.24352028965950012)]

In [5]:
QA_word2vec= Word2Vec.load("model_para/QA_word2vec.bin")
vocab = dict([(k, v.index) for k, v in QA_word2vec.wv.vocab.items()])
weight_matrix = QA_word2vec.wv.syn0 #word_to_vec

## label vector

In [51]:
max_len=0
for row in all_context:
    if len(row)>max_len:
        max_len = len(row)

print('max length of context: ',max_len)

answer_ptr = np.apply_along_axis(lambda x:(int(x[5]),int(x[5])+len(x[6])) , axis=1 , arr = train)
answer_ptr.shape , train.shape
np.save("../data/y_ptr.npy" , answer_ptr)

max length of context:  1158


In [49]:
i = 10
words = jieba.cut(train[i][2], cut_all=False)
a = []
for word in words:
    a.append(word)

len(a) , len(train[i][2])

(153, 316)

## jieba教學
http://blog.fukuball.com/ru-he-shi-yong-jieba-jie-ba-zhong-wen-fen-ci-cheng-shi/
  
需下載 dict.txt.big 以轉成繁體中文模式

In [65]:
import jieba

jieba.set_dictionary('../data/dict.txt.big')

content = open('lyric.txt', 'r').read()

print("Input：", content)

words = jieba.cut(content.split("\n")[0], cut_all=False)

print("Output 精確模式 Full Mode：")
lyric = []
for word in words:
    lyric.append(word)

Building prefix dict from /home/derricksu/Chinese_QA/data/dict.txt.big ...
Loading model from cache /tmp/jieba.u2c8973e8f27ca5fa986cc90b0a140f46.cache


Input： 哈囉你好嗎
我今天想吃牛排
而我聽見下雨的聲音
想起妳用唇語說愛情
今天不回家
只是個乾妹妹
你妹妹在哪
Output 精確模式 Full Mode：


Loading model cost 0.938 seconds.
Prefix dict has been built succesfully.


In [66]:
lyric

['哈囉', '你好', '嗎']

In [60]:
max_len=0
for para in train[:,2]:
    words = jieba.cut(para, cut_all=False)
    c = 0
    for word in words:
        c+=1
    if c>max_len:
        max_len = c
print("max length of paragraph after jieba :",max_len)

Building prefix dict from /home/derricksu/Chinese_QA/data/dict.txt.big ...
Dumping model to file cache /tmp/jieba.u2c8973e8f27ca5fa986cc90b0a140f46.cache
Loading model cost 1.313 seconds.
Prefix dict has been built succesfully.


In [32]:
import requests

res = requests.get("https://raw.githubusercontent.com/fxsjy/jieba/master/extra_dict/dict.txt.big")
res.text[0:100]

f = open("dict.txt.big","w")
f.write(res.text)
f.close()